<a href="https://colab.research.google.com/github/ShahrzadZolghadr/Sentiment-Analysis/blob/main/SA_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%load utils_prepro.py
import utils_prepro

In [18]:
import pandas as pd
import hazm
import re
from cleantext import clean
#import os
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [30]:
#!unzip /content/drive/MyDrive/snappfood

In [14]:
train = pd.read_csv('snappfood/train.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
test = pd.read_csv('snappfood/test.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
dev = pd.read_csv('snappfood/dev.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
train.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [15]:
train = train[['comment', 'label_id']]
test = test[['comment', 'label_id']]
dev = dev[['comment', 'label_id']]
train.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


## Preprocessing text column data

In [16]:
# calculate the length of comments based on their words
train['comment_len_by_words'] = train['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
min_max_len = train["comment_len_by_words"].min(), train["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 378


In [17]:
# cleaning comments
train['cleaned_comment'] = train['comment'].apply(utils_prepro.apply_cleaning)
test['cleaned_comment'] = test['comment'].apply(utils_prepro.apply_cleaning)
dev['cleaned_comment'] = dev['comment'].apply(utils_prepro.apply_cleaning)

train.head()

,comment,label_id,comment_len_by_words,cleaned_comment
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,9,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,28,قرار بود N ساعته برسه ولی نیم ساعت زودتر از مو...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,19,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,21,عالللی بود همه چه درست و به اندازه و کیفیت خوب...
4,شیرینی وانیلی فقط یک مدل بود.,0,7,شیرینی وانیلی فقط یک مدل بود


In [19]:
train = train[['cleaned_comment', 'label_id']]
test = test[['cleaned_comment', 'label_id']]
dev = dev[['cleaned_comment', 'label_id']]

x_train, y_train = train['cleaned_comment'].values.tolist(), train['label_id'].values.tolist()
x_test, y_test = test['cleaned_comment'].values.tolist(), test['label_id'].values.tolist()
x_dev, y_dev = dev['cleaned_comment'].values.tolist(), dev['label_id'].values.tolist()
print(train.shape)

(56700, 2)


# Keras and BiLSTM model

In [20]:
max_features = 20000  # Only consider the top 15k words
maxlen = 200  # Only consider the first 200 words of each movie review

In [21]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)
X_dev = tokenizer.texts_to_sequences(x_dev)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
X_dev = pad_sequences(X_dev, maxlen=maxlen)

# Build the model

In [22]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,757,761
Trainable params: 2,757,761
Non-trainable params: 0
_________________________________________________________________


In [24]:
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=1,monitor='val_loss'),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
]

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, np.array(y_train), batch_size=32, epochs=5, validation_data=(X_dev, np.array(y_dev)),callbacks=my_callbacks)

Epoch 1/5
1772/1772 [==============================] - 360s 197ms/step - loss: 0.3663 - accuracy: 0.8425 - val_loss: 0.3383 - val_accuracy: 0.8563
Epoch 2/5
1772/1772 [==============================] - 347s 196ms/step - loss: 0.2928 - accuracy: 0.8775 - val_loss: 0.3457 - val_accuracy: 0.8578


In [25]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, np.array(y_test), batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
219/219 [==============================] - 16s 74ms/step - loss: 0.3376 - accuracy: 0.8593
test loss, test acc: [0.3375803232192993, 0.8592857122421265]
Generate predictions for 3 samples
predictions shape: (3, 1)


# To check some samples

In [26]:
model.predict(pad_sequences(tokenizer.texts_to_sequences([['چقددددر آخه بیمزه باشه یه غذا']]),maxlen))

array([[0.34687603]], dtype=float32)

In [29]:
predictions # return probability of being HAPPY in comment

array([[0.9867191 ],
       [0.01921049],
       [0.7628948 ]], dtype=float32)